# IMPORT LIBRARIES

In [101]:
import os
import glob
import ROOT
from math import sqrt
import time
#from os import path
from pathlib import Path
import re
import copy
from tqdm.notebook import trange, tqdm
import numpy as np
from array import *
%jsroot on

# LOAD SPLINES AND DEFINE BINNING

In [2]:
35.9 + 41.5 + 59.7

137.10000000000002

In [3]:
lumi = {'MC_2016_CorrectBTag':35.9, 'MC_2017':41.5, 'MC_2018':59.7}

In [4]:
D2jetZHSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjZH_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
D2jetWHSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjWH_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
D2jetVBFSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjVBF_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

#WPCshift = 1
WPCshift2jv = 0.46386/(1. - 0.46386)
WPCshift2jz = 0.91315/(1. - 0.91315)
WPCshift2jw = 0.88384/(1. - 0.88384)

In [5]:
DbkgkinSpline4e = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_4e_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgkinSpline4mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_4mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgkinSpline2e2mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_2e2mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

DggbkgkinSpline4e = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_4e_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DggbkgkinSpline4mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_4mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DggbkgkinSpline2e2mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_2e2mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

In [6]:
DbkgjjEWQCDSpline4lHadVH = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_4l_HadVHTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgjjEWQCDSpline2l2lHadVH = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_2l2l_HadVHTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

DbkgjjEWQCDSpline4lJJVBF = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_4l_JJVBFTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgjjEWQCDSpline2l2lJJVBF = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_2l2l_JJVBFTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

In [24]:
medges = np.array([220, 230, 240, 250, 260, 280, 310, 340, 370, 400, 475, 550, 625, 700, 800, 900, 1000, 1200, 1600, 2000, 3000, 13000], dtype='float64')
# medges = np.array([220, 230, 240, 250, 260, 280, 310, 340, 370, 400, 475, 550, 625, 700, 800, 900, 1000, 1200, 1600, 2000, 13000], dtype='float64')
## medges = np.array([220, 260, 370, 625, 1000, 13000], dtype='float64')
## medges = np.array([220, 400, 3000, 13000], dtype='float64')
# medges = 5*np.arange(401, dtype='float64')
#d1edges = np.arange(21, dtype='float64') / 20 * 12780 + 220
##d1edges = np.array([ 220.,  316.,  412.,  508.,  604.,  700.,  796.,  892.,  988., 1084., 1180., 1276., 1372., 1468., 1564., 1660., 1756., 1852., 1948., 2044., 13000])
##d1edges = np.array([ 220.,  828., 1436., 2044.])
#d1edges = np.array([220, 4480, 8740, 13000], dtype='float64')
#d1edges = np.array([220, 13000], dtype='float64')
d1edges = np.arange(21, dtype='float64') / 20
#d1edges = np.arange(6, dtype='float64') / 5
# d1edges = np.arange(4, dtype='float64') / 3
# d1edges = np.arange(3, dtype='float64') / 2
d2edges = np.arange(21, dtype='float64') / 10 - 1
#d2edges = np.arange(6, dtype='float64') / 2.5 - 1
# d2edges = np.arange(4, dtype='float64') / 1.5 - 1
# d2edges = np.arange(3, dtype='float64') / 1 - 1

In [25]:
print("medges", len(medges))
print("medges", (medges))
print("d1edges", len(d1edges))
print("d1edges", (d1edges))
print("d2edges", len(d2edges))
print("d2edges", (d2edges))

medges 22
medges [  220.   230.   240.   250.   260.   280.   310.   340.   370.   400.
   475.   550.   625.   700.   800.   900.  1000.  1200.  1600.  2000.
  3000. 13000.]
d1edges 21
d1edges [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
d2edges 21
d2edges [-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


# CHOOSE INPUT TREES

In [9]:
#tree i made  "/eos/user/s/skyriaco/Offshell/200205_CutBased/"

### Read selected trees from user-specified file

In [12]:
# treelistpath = "/eos/user/l/lkang/Active_Research/Discriminants/alltaggedtrees.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/outputtree_list.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/alloutput.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/output_highmass1618.txt"
treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/output_highmass18.txt"
treelist = []

with open(treelistpath) as f:
    llist = [line.rstrip() for line in f]
        
for line in llist:
    if Path(line).exists(): treelist.append(line)

In [13]:
for t in treelist:
    #if "VBF" in t:
         print(t)

/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/ExtraIgnore/DYToLL_2J/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH155/ZZ4lAnalysis.root
/eo

In [14]:
yeardict = {}

for numfile in trange(len(treelist)):
    filename = treelist[numfile]
    ind = filename.split("/").index("200205_CutBased")
#     tagtreefile = "/".join(filename.split("/")[ind:])
    year = filename.split("/")[ind:][1]
#     print(year)
    if year not in yeardict.keys():
        yeardict[year] = {}
    else:
        if "ZZTo4l" in filename.split("/")[ind:][2]: prod = "ZZTo4l"
        else: prod = filename.split("/")[ind:][3]
            
        if prod not in yeardict[year]:
        #    if 'gg' in prod:
        #        yeardict[year][prod] = [[], [], []]
        #    else:
                yeardict[year][prod] = [[]]
        if 'gg' in prod:
            yeardict[year][prod][0].append(filename)
            #if 'ggTo2e2mu' in filename:
            #    yeardict[year][prod][0].append(filename)
            #elif 'ggTo4e' in filename:
            #    yeardict[year][prod][1].append(filename)
            #elif 'ggTo4mu' in filename:
            #    yeardict[year][prod][2].append(filename)
        elif "VBF" in prod:
            yeardict[year][prod][0].append(filename)
        elif "ZH" in prod:    
            yeardict[year][prod][0].append(filename)
        elif "Wplus"   in prod:    
            yeardict[year][prod][0].append(filename)  
        elif "Wminus"   in prod:    
            yeardict[year][prod][0].append(filename)   
        elif prod == "ZZTo4l":
            yeardict[year][prod][0].append(filename)
        else:
            print("ERROR: Cannot recognize production mode of " + filename + "! Tree not sorted!")

  0%|          | 0/155 [00:00<?, ?it/s]

### Check organized trees

In [15]:
yeardict

{'MC_2018': {'VBFH1000': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root']],
  'VBFH115': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root']],
  'VBFH135': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root']],
  'VBFH140': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root']],
  'VBFH145': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root']],
  'VBFH150': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root']],
  'VBFH1500': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root']],
  'VBFH155': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/

In [16]:
print (yeardict)

{'MC_2018': {'VBFH1000': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root']], 'VBFH115': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root']], 'VBFH135': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root']], 'VBFH140': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root']], 'VBFH145': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root']], 'VBFH150': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root']], 'VBFH1500': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root']], 'VBFH155': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMa

### Account for all bespoke EW MELA branches

In [17]:
refsam4l = ["VBFToContinToZZTo4l", "VBFToHiggs0L1ContinToZZTo4l", "VBFToHiggs0L1f025ph0ToZZTo4l", "VBFToHiggs0L1f05ph0ContinToZZTo4l", "VBFToHiggs0L1f05ph0ToZZTo4l", "VBFToHiggs0L1f075ph0ToZZTo4l", "VBFToHiggs0L1ToZZTo4l", "VBFToHiggs0MContinToZZTo4l", "VBFToHiggs0Mf025ph0ToZZTo4l", "VBFToHiggs0Mf05ph0ContinToZZTo4l", "VBFToHiggs0Mf05ph0ToZZTo4l", "VBFToHiggs0Mf075ph0ToZZTo4l", "VBFToHiggs0MToZZTo4l", "VBFToHiggs0PHContinToZZTo4l", "VBFToHiggs0PHf025ph0ToZZTo4l", "VBFToHiggs0PHf05ph0ContinToZZTo4l", "VBFToHiggs0PHf05ph0ToZZTo4l", "VBFToHiggs0PHf075ph0ToZZTo4l", "VBFToHiggs0PHToZZTo4l", "VBFToHiggs0PMContinToZZTo4l", "VBFToHiggs0PMToZZTo4l"]

In [18]:
refmel = ["p_Gen_JJEW_BKG_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1177p11_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m2038p82_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p164504_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p284929_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv4_0p216499_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p157323_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p272492_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv2_0p207049_MCFM", "p_Gen_JJEW_BSI_ghv1_1_MCFM", "p_Gen_JJEW_SIG_ghv1_1_MCFM"]

In [19]:
refden = dict(zip(refsam4l, refmel))

In [20]:
refden

{'VBFToContinToZZTo4l': 'p_Gen_JJEW_BKG_MCFM',
 'VBFToHiggs0L1ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_0_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f025ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1177p11_MCFM',
 'VBFToHiggs0L1f05ph0ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_1_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f05ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f075ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m2038p82_MCFM',
 'VBFToHiggs0L1ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_0_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0MContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_0_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf025ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p164504_MCFM',
 'VBFToHiggs0Mf05ph0ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_1_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf05ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf075ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p284929_MCFM',
 'VBFToHiggs0MToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_0_ghv4_0p216499_MCFM',
 'V

# REPEAT THIS SECTION AS NEEDED TO CREATE TEMPLATES AND FILL 'hlist'

##### Note that 'hlist' is expected to look like ['ggH SIG', 'ggH BSI', 'ggH BKG', 'EW SIG', 'EW BSI', 'EW BKG', 'qqbar BKG'] so make the appropriate histograms then append to 'hlist'
##### As you change the year, production mode, final state, SIG/BKG/INT mix, and category, you should re-run the 'Fill histogram' cell and check the output before adding the next template to our 'hlist'

### Set up basic information. Make list to store histograms---do not evaluate this cell unless you want an empty instance of hlist!

In [38]:
hlist = ['ggH SIG', 'ggH BSI', 'ggH BKG', 'qqbar BKG']

In [6]:
targetreweight = {"gg": ["p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal", "p_Gen_GG_BSI_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal", "p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal"],
                 "VBF": ["p_Gen_JJEW_SIG_ghv1_1_MCFM*0.5", "p_Gen_JJEW_BSI_ghv1_1_MCFM*0.5", "p_Gen_JJEW_BKG_MCFM*0.5"],
                 "ZZTo4l": ["1"]}

finalstate = {"4e": "121*121", "4mu": "169*169", "2e2mu": "121*169"}

In [26]:
proc = ["ggH_0PM", "ggH_g11g21_negative", "ggH_g11g21_positive", "back_ggZZ", "back_qqZZ", "data_obs"]

In [ ]:
#"qqH_0PM", "qqH_g12g22_negative", "qqH_g12g22_positive", "back_VVZZ"

In [ ]:
print year

### Choose year, production mode, final state, SIG/BKG/INT mix, and category

In [21]:
#configure what you would like to run

ltargetyear = ["MC_2018"] #,"MC_2016_CorrectBTag","MC_2018"]
ltargetprod = ["gg"] #,"VBF","ZZTo4l"]
ltargetstate = ["4l"] #"4mu","4e","2e2mu"]
ltargetcomp = [0,1,2]    # SIG=0 BSI=1 BKG=2    (for the qqbar bkg events in 'ZZTo4l' this choice does not matter as you can see above)
ltargetcateg = [0,1,2]  # Untag=0 VBF=1 VH=2

### Fill histogram

In [116]:
#syst_weights = []



def FillHist(targetprod,targetcomp,targetcateg,h_list) :

    print (targetreweight[targetprod][targetcomp])
    print (targetprod,targetcomp,targetcateg)
    weight = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag=={})*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp], targetcateg)
    weightewzzup = "{}*(1.0 + KFactor_EW_qqZZ_unc)*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag=={})*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp], targetcateg)
    weightewzzdn = "{}*(1.0 - KFactor_EW_qqZZ_unc)*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag=={})*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp], targetcateg)
    
    weightqrup = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag=={})*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp], targetcateg)
    weightqrdn = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag=={})*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp], targetcateg)

    
    LHEweightsUp = ["LHEweight_QCDscale_muR2_muF1","LHEweight_QCDscale_muR1_muF2","LHEweight_PDFVariation_Up","LHEweight_AsMZ_Up","PythiaWeight_isr_muR4 * PythiaWeight_fsr_muR4"]
    LHEweightsDn = ["LHEweight_QCDscale_muR0p5_muF1","LHEweight_QCDscale_muR1_muF0p5","LHEweight_PDFVariation_Dn","LHEweight_AsMZ_Dn","PythiaWeight_isr_muR0p25 * PythiaWeight_fsr_muR0p25"]
    
    
    hf_LHEup = []
    hf_LHEdn = []
    
    for i,w_ght in enumerate(LHEweightsUp) : 
          LHEweightsUp[i] = w_ght+"*"+weight
          print (LHEweightsUp[i])  
          LHEweightsDn[i] = LHEweightsDn[i]+"*"+weight
          h_temp = ROOT.TH1F("hf_temp","",len(medges)-1, medges)
          h_tempp = copy.deepcopy(h_temp)  
          hf_LHEup.append(h_tempp)
          hf_LHEdn.append(h_tempp)
 
   
    hf = ROOT.TH3F("hf","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
    hf_systdn = ROOT.TH1F("hf_kew_systdn","",100,0,2000)
    hf_systup = ROOT.TH1F("hf_kew_systup","",100,0,2000)
    hf_nom    = ROOT.TH1F("hf_nom","",100,0,2000)
    hf_nomlhe = ROOT.TH1F("hf_nomlhe","",len(medges)-1, medges)    
    hf_qr_up  = ROOT.TH1F("hf_qr_up","",100,0,2000)
    hf_qr_dn  = ROOT.TH1F("hf_qr_dn","",100,0,2000)    
    
    
    hf.SetDirectory(0)
    hf_systdn.SetDirectory(0)
    hf_systup.SetDirectory(0)
    hf_nom.SetDirectory(0)
    hf_nomlhe.SetDirectory(0)
    for i,ht in enumerate(LHEweightsUp) :              
            hf_LHEup[i].SetDirectory(0)
            hf_LHEdn[i].SetDirectory(0)

    for keynum in trange(len(yeardict.keys())):
        year = list(yeardict.keys())[keynum]
        #if year != targetyear: continue
        #print("\n", year, lumi[year])

        hs = ROOT.TH3F("hs","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
        hs_systdn = ROOT.TH1F("hs_kew_systdn","",100,0,2000)
        hs_systup = ROOT.TH1F("hs_kew_systup","",100,0,2000)
        hs_nom    = ROOT.TH1F("hs_nom","",100,0,2000)
        hs_nomlhe = ROOT.TH1F("hs_nomlhe","",len(medges)-1, medges)

        hs_LHEup = []
        hs_LHEdn = []        
        for i,ht in enumerate(hf_LHEup) :   
            hs_temp = ROOT.TH1F("hs_temp","",len(medges)-1, medges)
            hs_temp.SetDirectory(0)
            hs_tempp = copy.deepcopy(hs_temp) 
            hs_LHEup.append(hs_tempp)
            hs_LHEdn.append(hs_tempp)
        
        
        hs_qr_up  = ROOT.TH1F("hs_qr_up","",100,0,2000)
        hs_qr_dn  = ROOT.TH1F("hs_qr_dn","",100,0,2000)
        
        hs.SetDirectory(0)
        hs_systdn.SetDirectory(0)
        hs_systup.SetDirectory(0)
        hs_nom.SetDirectory(0)
        hs_nomlhe.SetDirectory(0)
        for i,ht in enumerate(hf_LHEup) :              
            hs_LHEup[i].SetDirectory(0)
            hs_LHEdn[i].SetDirectory(0)
            
        for prodd in yeardict[year]: 
          if targetprod in prodd :   
           for decay in yeardict[year][prodd]:
            #         if "2e2mu" not in decay[0]: continue
            #print("\n", decay)
            ht = ROOT.TH3F("ht","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
            ht_systdn = ROOT.TH1F("ht_kew_systdn","",100,0,2000)
            ht_systup = ROOT.TH1F("ht_kew_systup","",100,0,2000)
            ht_nom    = ROOT.TH1F("ht_nom","",100,0,2000)
            ht_nomlhe = ROOT.TH1F("ht_nomlhe","",len(medges)-1, medges)
            
        
            ht_qr_up  = ROOT.TH1F("hs_qr_up","",100,0,2000)
            ht_qr_dn  = ROOT.TH1F("hs_qr_dn","",100,0,2000)
            
            ht_LHEup = []
            ht_LHEdn = []        
            for i,htf in enumerate(hf_LHEup) :   
                ht_temp = ROOT.TH1F("ht_temp","",len(medges)-1, medges)
                ht_temp.SetDirectory(0)
                ht_tempp = copy.deepcopy(ht_temp) 
                ht_LHEup.append(ht_tempp)
                ht_LHEdn.append(ht_tempp)
                
                
            ht.SetDirectory(0)
            ht_systdn.SetDirectory(0)
            ht_systup.SetDirectory(0)
            ht_nom.SetDirectory(0)
            ht_nomlhe.SetDirectory(0)
            ht_qr_up.SetDirectory(0)  
            ht_qr_dn.SetDirectory(0)  
            for i,htff in enumerate(ht_LHEup) :   
            
                ht_LHEup[i].SetDirectory(0)
                ht_LHEdn[i].SetDirectory(0)
            count = 0
            
            
            for tfile in trange(len(decay)):

                print(decay[tfile])
                if "VBF" in decay[tfile]:

                    #if "0L1" in decay[tfile]: continue
                    skey = decay[tfile].split("/")[-2].replace('_M125_GaSM', '')
                    sampleweight = refden[skey]
    #                 print(sampleweight)
                if ( os.stat(decay[tfile]).st_size > 100 ):  
                        f = ROOT.TFile(decay[tfile])
                        t = f.Get("eventTree")
                        htt = ROOT.TH3F("htt","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
                        htt_ew_systdn = ROOT.TH1F("htt_ew_systdn","",100,0,2000)
                        htt_ew_systup = ROOT.TH1F("htt_ew_systup","",100,0,2000)
                        htt_nom    = ROOT.TH1F("htt_nom","",100,0,2000)
                        htt_nomlhe = ROOT.TH1F("htt_nomlhe","",len(medges)-1, medges)

                        htt_LHEup = []
                        htt_LHEdn = []        
                
                        for i,htff in enumerate(ht_LHEup) :
                            print (LHEweightsUp[i])
                            htt_tempu = ROOT.TH1F("htt_tempu","",len(medges)-1, medges)
                            t.Draw("ZZMass>>htt_tempu",LHEweightsUp[i],"")                                
                            htt_LHEup.append(htt_tempu)
                            print (LHEweightsDn[i])
                            print ("----------------------")
                            htt_tempd = ROOT.TH1F("htt_tempd","",len(medges)-1, medges)
                            t.Draw("ZZMass>>htt_tempd",LHEweightsDn[i],"")
                            htt_LHEdn.append(htt_tempd)
                            print ("up:",htt_tempu.Integral())
                            print ("dn:",htt_tempd.Integral())
                            htt_LHEup[i].SetDirectory(0)
                            htt_LHEdn[i].SetDirectory(0)

                        htt_qr_up  = ROOT.TH1F("hs_qr_up","",100,0,2000)
                        htt_qr_dn  = ROOT.TH1F("hs_qr_dn","",100,0,2000)
                        #print (weight_ewzz_up)
                        #print (weight_ewzz_dn)
                        #print (weight)

                        t.Draw("Dbsi:Dbkg:ZZMass>>htt",weight,"")                        
                        t.Draw("ZZMass>>htt_nom",weight,"")
                        t.Draw("ZZMass>>htt_nomlhe",weight,"")
                        print ("om nomnomm:",htt_nomlhe.Integral())
                        t.Draw("ZZMass>>htt_ew_systup",weightewzzup,"")
                        t.Draw("ZZMass>>htt_ew_systdn",weightewzzdn,"")

                        
                        t.Draw("ZZMass>>htt_qr_up",weightqrup,"")    
                        t.Draw("ZZMass>>htt_qr_dn",weightqrdn,"")    


                        #print ("htt_up:", htt_ew_systup.Integral())
                        #print ("htt_dn:", htt_ew_systdn.Integral())
                        
                        if count == 0:
                            #print("ADDED --- htt = ", htt.Integral())
                            ht.Add(htt)
                            ht_nom.Add(htt_nom)
                            
                            #print (htt_nomlhe)
                            #print (ht_nomlhe)
                            ht_nomlhe.Add(htt_nomlhe)

                            ht_systdn.Add(htt_ew_systdn)
                            ht_systup.Add(htt_ew_systup)
                            
                            ht_qr_up.Add(htt_qr_up)
                            ht_qr_dn.Add(htt_qr_dn)
                            
                            for i,htff in enumerate(ht_LHEup) :   
            
                                ht_LHEup[i].Add(htt_LHEup[i])
                                ht_LHEdn[i].Add(htt_LHEdn[i])
                                print ("here",ht_LHEup[i].Integral())
                                print ("here",ht_LHEdn[i].Integral())

                            
                            count+=1
                        elif htt.Integral() > ((ht.Integral()/count) + 2*sqrt((ht.Integral()/count))): 
                            print("SKIPPED --- htt = ", htt.Integral())
                        elif htt.Integral() < ((ht.Integral()/count) - 2*sqrt((ht.Integral()/count))): 
                            print("SKIPPED --- htt = ", htt.Integral())
                        else:
                            print("ADDED --- htt = ", htt.Integral())
                            ht.Add(htt)
                            ht_nom.Add(htt_nom)
                            ht_nomlhe.Add(htt_nomlhe)

                            ht_systdn.Add(htt_ew_systdn)
                            ht_systup.Add(htt_ew_systup)
                            for i,htff in enumerate(ht_LHEup) : 
                                ht_LHEup[i].SetDirectory(0)
                                ht_LHEdn[i].SetDirectory(0)
                                print ("sfin:",ht_LHEup[i].Integral())
                                print ("sfindn:",ht_LHEdn[i].Integral())
                                ht_LHEup[i].Add(htt_LHEup[i])
                                ht_LHEdn[i].Add(htt_LHEdn[i])
                                            
                            ht_qr_up.Add(htt_qr_up)
                            ht_qr_dn.Add(htt_qr_dn)    
                            
                            count+=1
            #print("ht = ", ht.Integral())
            ht.Scale(1./count)
            ht_nom.Scale(1./count)
            ht_systdn.Scale(1./count)
            ht_systup.Scale(1./count)
            
            for i,htff in enumerate(ht_LHEup) :            
                ht_LHEup[i].Scale(1./count)
                ht_LHEdn[i].Scale(1./count)        
            #print("ht = ", ht.Integral())
            hs.Add(ht)
            hs_systdn.Add(ht_systdn)
            hs_systup.Add(ht_systup)
            hs_nom.Add(ht_nom)
            hs_nomlhe.Add(ht_nomlhe)

            hs_qr_up.Add(ht_qr_up)
            hs_qr_dn.Add(ht_qr_dn) 
            for i,htff in enumerate(ht_LHEup) :            
                hs_LHEup[i].Add(ht_LHEup[i])
                hs_LHEdn[i].Add(ht_LHEdn[i])
            #print("hs = ", hs.Integral())

        hf.Add(hs)
        hf_systdn.Add(hs_systdn)
        hf_systup.Add(hs_systup)
        hf_nom.Add(hs_nom)
        hf_nomlhe.Add(hs_nomlhe)
        for i,htff in enumerate(hf_LHEup) : 

                hf_LHEup[i].Add(hs_LHEup[i])
                hf_LHEdn[i].Add(hs_LHEdn[i])
                
        hf_qr_up.Add(hs_qr_up)
        hf_qr_dn.Add(hs_qr_dn) 
        print("hf = ", hf_nom.Integral())

        hf.Scale(1/len(yeardict.keys()))
        hf_systdn.Scale(1/len(yeardict.keys()))
        hf_systup.Scale(1/len(yeardict.keys()))
        hf_nom.Scale(1/len(yeardict.keys()))
        
        for i,htff in enumerate(hf_LHEup) :            
                hf_LHEup[i].Scale(1/len(yeardict.keys()))
                hf_LHEdn[i].Scale(1/len(yeardict.keys()))
                h_list.append(hf_LHEup[i])
                h_list.append(hf_LHEdn[i])


        #h_list.append(hf)
        #h_list.append(hf_systdn)
        #h_list.append(hf_systup)
        h_list.append(hf_nomlhe)
        

In [117]:

h_gg_unt =[]
print (h_gg_unt)
for tcomp in ltargetcomp: 
       FillHist("gg",tcomp,0,h_gg_unt)
'''
h_gg_vbf =[]
for tcomp in ltargetcomp: 
       FillHist("gg",tcomp,1,h_gg_vbf)

h_gg_vh =[]
for tcomp in ltargetcomp: 
       FillHist("gg",tcomp,2,h_gg_vh)     
        
 
       
#=========================================  
h_ew_unt =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,0,h_ew_unt)

h_ew_vbf =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,1,h_ew_vbf)

h_ew_vh =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,2,h_ew_vh)      
              
#=========================================  
h_bkg_unt =[]
FillHist("ZZTo4l",0,0,h_bkg_unt)

h_bkg_vbf =[]
FillHist("ZZTo4l",0,1,h_bkg_vbf)
    
h_bkg_vh =[]
FillHist("ZZTo4l",0,2,h_bkg_vh)      
'''       

[]
p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal
gg 0 0
LHEweight_QCDscale_muR2_muF1*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR1_muF2*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_PDFVariation_Up*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_AsMZ_Up*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/ggH1000/ZZ4lAnalysis.root
LHEweight_QCDscale_muR2_muF1*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR0p5_muF1*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
----------------------
up: 107.90133531740867
dn: 139.28114393958822
LHEweight_QCDscale_muR1_muF2*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCD

  0%|          | 0/1 [00:00<?, ?it/s]

...

/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/ggH800/ZZ4lAnalysis.root
LHEweight_QCDscale_muR2_muF1*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR0p5_muF1*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
----------------------
up: 704.4772500350373
dn: 934.0669193069916
LHEweight_QCDscale_muR1_muF2*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR1_muF0p5*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1F

  0%|          | 0/1 [00:00<?, ?it/s]

/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/ggH900/ZZ4lAnalysis.root
LHEweight_QCDscale_muR2_muF1*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR0p5_muF1*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
----------------------
up: 789.4523211971391
dn: 1037.94131213543
LHEweight_QCDscale_muR1_muF2*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*(EventTag==0)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR1_muF0p5*p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Fl

'\nh_gg_vbf =[]\nfor tcomp in ltargetcomp: \n       FillHist("gg",tcomp,1,h_gg_vbf)\n\nh_gg_vh =[]\nfor tcomp in ltargetcomp: \n       FillHist("gg",tcomp,2,h_gg_vh)     \n        \n \n       \n#=========================================  \nh_ew_unt =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,0,h_ew_unt)\n\nh_ew_vbf =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,1,h_ew_vbf)\n\nh_ew_vh =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,2,h_ew_vh)      \n              \n#=========================================  \nh_bkg_unt =[]\nFillHist("ZZTo4l",0,0,h_bkg_unt)\n\nh_bkg_vbf =[]\nFillHist("ZZTo4l",0,1,h_bkg_vbf)\n    \nh_bkg_vh =[]\nFillHist("ZZTo4l",0,2,h_bkg_vh)      \n'

Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hs_qr_up (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hs_qr_dn (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_up (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_dn (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_up (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_dn (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_up (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hs_qr_

In [94]:
c1 = ROOT.TCanvas("c1","c1",1800,600)
#c1.Divide(3,1)


print (len(h_gg_unt))
for i in range(0,len(h_gg_unt)): 
    h_gg_unt[i].SetLineStyle(2)
h_gg_unt[32].SetLineStyle(1)
h_gg_unt[32].SetLineColor(2)
h_gg_unt[2].SetLineColor(4)
h_gg_unt[3].SetLineColor(4)

c1.cd().SetLogy()

h_gg_unt[32].Draw()
print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
h_gg_unt[0].Draw("same")
h_gg_unt[1].Draw("same")

h_gg_unt[2].Draw("same")
h_gg_unt[3].Draw("same")

c1.Draw()


33
51023.70595741272 50519.019906044006 50519.019906044006


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


### Add to 'hlist' if the output looks reasonable above

In [67]:
c1 = ROOT.TCanvas("c1","c1",1800,600)
c1.Divide(3,1)
h_bkg_unt[3].SetLineColor(2)
h_bkg_vh[3].SetLineColor(2)
h_bkg_vbf[3].SetLineColor(2)

h_bkg_unt[1].SetLineStyle(2)
h_bkg_vh[1].SetLineStyle(2)
h_bkg_vbf[1].SetLineStyle(2)


h_bkg_unt[2].SetLineStyle(2)
h_bkg_vh[2].SetLineStyle(2)
h_bkg_vbf[2].SetLineStyle(2)


c1.cd(1).SetLogy()
#c1.cd(1)

print (h_bkg_unt[1].Integral())
h_bkg_unt[3].Draw("hist")
h_bkg_unt[3].GetXaxis().SetTitle("qqZZ Untagged background m4l [GeV]")
h_bkg_unt[1].Draw("hist,same")
h_bkg_unt[2].Draw("hist,same")
c1.cd(2).SetLogy()
#c1.cd(2)
h_bkg_vh[3].Draw("hist")
h_bkg_vh[3].GetXaxis().SetTitle("qqZZ VHtagged background m4l [GeV]")
h_bkg_vh[1].Draw("hist,same")
h_bkg_vh[2].Draw("hist,same")



c1.cd(3).SetLogy()
#c1.cd(3)
h_bkg_vbf[3].Draw("hist")
h_bkg_vbf[3].GetXaxis().SetTitle("qqZZ VBFtagged background m4l [GeV]")
h_bkg_vbf[1].Draw("hist,same")
h_bkg_vbf[2].Draw("hist,same")


ROOT.gStyle.SetOptStat(1)
c1.Draw()
ROOT.gStyle.SetOptStat(1)
c1.Update()

c1.SaveAs("back_ew_qqzz.png")
c1.SaveAs("back_ew_qqzz.pdf")


NameError: name 'h_bkg_unt' is not defined

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [ ]:
print  (h_gg_unt)

In [ ]:
print (h_list)

In [ ]:
hlist

In [ ]:
for h in h_list:
    print (h)
    print(h.Integral())

# Unroll and Visualize all histograms in 'hlist'

##### Do not proceed if your 'hlist' is not filled. Recall it expects 7 histograms ordered like: ['ggH SIG', 'ggH BSI', 'ggH BKG', 'EW SIG', 'EW BSI', 'EW BKG', 'qqbar BKG']
##### If you simply want to test the unrolling section, just fill the list with 7 copies of the same TH3F histogram

### Unroll all 7 histograms

In [ ]:
hlist1d = []
hlisttemp = h_list
for hf in hlisttemp:
    totgbins = hf.GetNbinsX() * hf.GetNbinsY() * hf.GetNbinsZ()
    totgedges = np.arange(totgbins+1, dtype='float64')

    #hf1d = ROOT.TH1F("hf1d","",len(totgedges)-1, totgedges)
    hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
    hf1d.SetDirectory(0)

    bincount = 1

    for xbin in trange(hf.GetNbinsX()):
        for ybin in range(hf.GetNbinsY()):
            for zbin in range(hf.GetNbinsZ()):
                gbin = hf.GetBin(xbin+1, ybin+1, zbin+1)

                #if gbin<400: print(xbin, ybin, zbin)
                #print(gbin)
                #print(bincount)
                if bincount>totgbins: 
                    print("WHAT")
                    break
                    
                #if hf.GetBinContent(gbin) != 0: print(hf.GetBinContent(gbin))
                #print(hf.GetBinContent(gbin))
                
                hf1d.SetBinContent(bincount, hf.GetBinContent(gbin))
                bincount += 1
                #if hf.GetBinContent(gbin) != 0: print("YAY")
    
    hlist1d.append(hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

### Isolate INT from the BSI histograms and also add the total combination of statistics as a new histogram at the end

In [ ]:
hlist1d[1].Add(hlist1d[0], -1)
hlist1d[1].Add(hlist1d[2], -1)

hlist1d[4].Add(hlist1d[3], -1)
hlist1d[4].Add(hlist1d[5], -1)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.SetDirectory(0)
hf1d.Add(hlist1d[1])
hlist1d.insert(1, hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.SetDirectory(0)
hf1d.Add(hlist1d[5])
hlist1d.insert(5, hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
for i in range(hlist1d[1].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[1].GetBinContent(b) >= 0:
        hlist1d[1].SetBinContent(b, 0)

for i in range(hlist1d[2].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[2].GetBinContent(b) <= 0:
        hlist1d[2].SetBinContent(b, 0)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
for i in range(hlist1d[5].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[5].GetBinContent(b) >= 0:
        hlist1d[5].SetBinContent(b, 0)

for i in range(hlist1d[6].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[6].GetBinContent(b) <= 0:
        hlist1d[6].SetBinContent(b, 0)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
for i in range(len(hlist1d)):
    hf1d.Add(hlist1d[i])
hlist1d.append(hf1d)
print(hf1d.Integral())

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hlist1d[1].Scale(-1)
hlist1d[5].Scale(-1)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.Add(hlist1d[-1])

In [ ]:
hlist1d.append(hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1dt = hlist1d.pop(-1)

In [ ]:
for i in range(hlist1d[-1].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[-1].GetBinContent(b) <= 0:
        hlist1d[-1].SetBinContent(b, 0)

### Check the final template collection and yields for the datacard

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

# Saving the whole thing to a .root file for use with a corresponding datacard

### Checking the names used to save the histograms

In [ ]:
for i in proc:
    print(i)

### Set directory and output file names

In [ ]:
diname = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/Output/"
foname = "Cat_4mu_137p1invfb_ggH4landVBF4landqqbar_mostEW_RECOpassedevents_mvar21d1reg20d2reg20.input.root"

In [ ]:
print(diname+foname)

### Write to a .root file

In [ ]:
fo = ROOT.TFile(diname+foname,"CREATE")

for i in range(len(proc)):
    hlist1d[i].Write(proc[i], ROOT.TFile.kSingleKey)

fo.Close()

### Check the saved file

In [ ]:
fo = ROOT.TFile(diname+foname)
# fo = ROOT.TFile("/eos/user/l/lkang/Active_Research/Templates/SManalysis_split/2017/Untag/4mu/Untag_4mu_137p1invfb_ggH4landVBF4landqqbar_mostEW_RECOpassedevents_mvar21d1reg20d2reg20.input.root")

fo.ls()

c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(proc),1)
for i in range(len(proc)):
    c1.cd(i+1)
    eval("print(proc[i], '\t\t=', fo."+proc[i]+".Integral())")
#     eval("print(fo."+proc[i]+".Integral())")
    eval("fo."+proc[i]+".Draw('hist')")
print()    

c1.Draw()
fo.Close()